# BoilingData Demo Notebook

## Install requirements and run init code

First we install py-boilingdata. Also pandas and numpy for getting Data Frame capabilities.

In [1]:
pip install git+https://github.com/boilingdata/py-boilingdata@v0.2.10 pandas numpy

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/boilingdata/py-boilingdata (to revision v0.2.9) to /private/var/folders/vy/p7dz__m15lb1pbm6xk1b9gxw0000gn/T/pip-req-build-nfbjf2sn
  Running command git clone --filter=blob:none --quiet https://github.com/boilingdata/py-boilingdata /private/var/folders/vy/p7dz__m15lb1pbm6xk1b9gxw0000gn/T/pip-req-build-nfbjf2sn
  Running command git checkout -q c776c2c632de06c3e3c10d503eed8cbf1b954f90
  Resolved https://github.com/boilingdata/py-boilingdata to commit c776c2c632de06c3e3c10d503eed8cbf1b954f90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for py-boilingdata: filename=py_boilingdata-0.2.9-py3-none-any.whl size=6382 sha256=602279f130d6a72a26f052b590454edde69eeb6a32609ebbea0e87bd7a509fc3
  Stored in directory: /private/var/folders/vy/p7dz__m15lb1pbm6xk1b9gxw0000gn/T/pip-ephem-wheel-ca

Import the needed packages into the python context and create `bd()` method for calling BoilingData. We initialise with log_level `INFO` (default) so that we also get some query runtime info. 

**`BD_USERNAME` and `BD_PASSWORD` are loaded from the `.env` file with `dotenv` package into the environment variables. BoilingData will read them from there and use them for login.**

The `bd()` method tries to convert the results into DF but if it fails, just returns the results as is (array of JSON).

Once you execute the code below, web socket connection is made.

In [10]:
import pandas as pd
from dotenv import load_dotenv
from py_boilingdata import BoilingData

load_dotenv()  # take environment variables from .env

# You can turn off the messages by raising the log level to 'ERROR'
boiling = BoilingData(log_level="ERROR")
await boiling.connect()

async def bd(sql):
    resp = await boiling.execute(sql)
    try:
        return pd.DataFrame(resp)
    except:
        print(resp)
        return resp

## Data Sets shared to/from you

To get a list of Data Sets shared to you, you can call the code below. It may be that you don't have any data sets shared to you or you have not shared any for others (no results).

In [8]:
await bd("SELECT shareName, fromEmail, lifeTime, schedule FROM boilingshares LIMIT 100;")

,shareName,fromEmail,lifeTime,schedule
0,taxi_locations,demo@boilingdata.com,1h,* * * * * *
1,taxi_locations_limited,dforsber@gmail.com,1h,* * * * * *
2,taxi_locations,dforsber@gmail.com,1h,* * * * * *
3,demo_full,dforsber@gmail.com,1h,* * * * * *


In this run, I have data set called `taxi_locations` shared for me by `dforsber@gmail.com`. I can query it like this: 

In [9]:
await bd("SELECT * FROM share('dforsber@gmail.com:taxi_locations') LIMIT 2")

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone


## Small data set example

The `test.parquet` is a small Parquet file, we can query as well.

In [11]:
await bd("SELECT * FROM parquet_scan('s3://boilingdata-demo/test.parquet') LIMIT 5")

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29+00,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03+00,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31+00,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21+00,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31+00,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,


## Listing S3

We can also list S3.

In [12]:
await bd("SELECT * FROM list('s3://')")

,Name,CreationDate
0,boilingdata-demo,2021-12-06T12:06:12.060Z
1,boilingdata-user,2021-12-06T12:06:12.060Z


In [13]:
await bd("SELECT * FROM list('s3://boilingdata-demo/');")

,Key,LastModified,ETag,Size,StorageClass,Name
0,demo.parquet,2022-06-18T10:14:03.000Z,"""f5d2e2bda78a61d9ed9a184ccf3beba2-58""",484530996,STANDARD,NaN
1,demo.small.parquet,2022-10-26T04:57:14.000Z,"""bc1b400cc0ebe3194e1fa6111d09c853-21""",172001116,STANDARD,NaN
2,demo2.parquet,2022-06-18T10:14:24.000Z,"""f5d2e2bda78a61d9ed9a184ccf3beba2-58""",484530996,STANDARD,NaN
3,hits.medium.parquet,2023-09-26T13:50:13.000Z,"""1b6504d05a7ddfa4375a8381f59c9596-98""",820431621,STANDARD,NaN
4,hits.parquet,2023-09-26T05:38:26.000Z,"""359976a01ddb98f2363d076a276acb24-1762""",14779976446,STANDARD,NaN
5,hits.small.parquet,2023-09-26T11:18:52.000Z,"""75786c21f574274fd58a103593a6d642-8""",64783825,STANDARD,NaN
6,taxi_locations.parquet,2023-04-23T13:35:21.000Z,"""58e27826358dcaf2ce59eb5908f73e89""",6168,STANDARD,NaN
7,test.duckdb,2023-11-10T07:58:59.000Z,"""18afc6620c3ab81c21482f633c430836""",536576,STANDARD,NaN
8,test.parquet,2022-05-23T16:37:00.000Z,"""19c7dc463166dd08c931736ad9048a35""",2783,STANDARD,NaN
9,test2.parquet,2023-02-25T20:41:24.000Z,"""425059579ebe794e8260c4f0d2173221""",2967,STANDARD,NaN


In [ ]:
await boiling.close()